In [1]:
import DeepLabModel
import os, urllib
from PIL import Image
from io import BytesIO
import numpy as np
from skimage.io import imshow
import matplotlib.pyplot as plt
from matplotlib import gridspec
from ipywidgets import FloatSlider, interact, fixed, HBox, VBox, Label, Button, Output, IntProgress, FloatProgress
%pdb 0
plt.rcParams['figure.max_open_warning'] = False
import pandas as pd
import IPython
%matplotlib inline
from skimage.measure import label

Automatic pdb calling has been turned OFF


### Combining Segmentation models using Cooperative Negotiation

In [2]:
class Agent():
    def __init__(self, agentname, model, alpha):
        self.agentname=agentname
        self.model=model
        self.task = None
        self.optimal = None
        self.alpha = alpha
        
    def new_task(self, image):
        self.task, self.optimal = self.model.run(image)
        print(self.optimal.shape)
        return self.optimal, self.utility(self.optimal)
    
    def _dice_score(x, y):
        x_and_y = np.count_nonzero(np.logical_and(x, y))
        card_x = np.count_nonzero(x)
        card_y = np.count_nonzero(y)
        return 2*x_and_y/(card_x+card_y+1e-16)
    
    def utility(self, proposal):
        # Since we are using the smooth dice loss, we have to split each label in the segmentation and sum up their scores
        dist =lambda x,y: 1.-Agent._dice_score(x,y)
        labels = np.unique(proposal)
        util = np.sum([dist(proposal==l, self.optimal==l) for l in labels])
        return util
    
    def propose(self, agreement):
        negotiation_map = np.zeros_like(agreement[0]) # For keeping track on what pixels are still to negotiate
        proposal = np.zeros_like(agreement[0]) # Initialize an empty proposal
        for l, conf in agreement.items(): # For each "probability map" received in the agreement
                proposal[conf==1] = l
                negotiation_map[conf==1] = 1

        x, y=np.where(negotiation_map!=1)
        
        
        
        for x, y in zip(x, y):
            # For each pixel, pick among the possible labels with the relative probability
            labels = list(agreement.keys())
            probs = [agreement[l][x,y] for l in labels]
            belief = self.optimal[x,y] # The agent thinks that the right label is this one
            if np.random.random() < self.alpha[belief]:
                proposal[x,y] = belief
            else:
                proposal[x,y] = np.random.choice(labels, p=probs)

                
                
        proposal = proposal.astype(np.int)
        utility = self.utility(proposal)   
        return proposal, utility

In [3]:
class Mediator():
    def __init__(self, agents):
        self.agents = agents
        self.last_step=0
        
    def start_new_task(self, image):
        self.task = image
        received_proposals = [agent.new_task(self.task) for agent in self.agents]
        self.initial_proposals = np.array([a[0] for a in received_proposals]) # Shape (agents, h, w)
        utilities_array = np.array([a[1] for a in received_proposals]) # Shape (agents, )
        self.initial_utilities = np.transpose(np.tile(utilities_array, [self.initial_proposals.shape[1], self.initial_proposals.shape[2], 1]), [2,0,1]) # Shape (agents, h, w)
        self.last_proposals = self.initial_proposals
        self.last_utilities = self.initial_utilities
        return self.last_proposals, self.last_utilities          

    def negotiation(self, timeout = 1000):
        for i in range(self.last_step, self.last_step+timeout):
            self.last_step = i
            # Agreement must be built label-wise (averaging over the values could produce new labels)
            
            all_proposed_labels = np.unique(self.last_proposals)
            agreements = {l:None for l in all_proposed_labels}
            for l in all_proposed_labels:
                prop = (self.last_proposals==l)
                util = (self.last_utilities==l)
                agreement_l = np.sum(prop*(1+util), axis=0)/np.sum(1+util, axis=0)
                agreements[l] = agreement_l
            
            
            # Propose the new agreement to the agents
            received_proposals = [agent.propose(agreements) for agent in self.agents]
            
            
            self.last_proposals = np.array([a[0] for a in received_proposals]) # Shape (agents, h, w)
            utilities_array = np.array([a[1] for a in received_proposals]) # Shape (agents, )
            self.last_utilities = np.transpose(np.tile(utilities_array, [self.last_proposals.shape[1], self.last_proposals.shape[2], 1]), [2,0,1]) # Shape (agents, h, w)
            
            yield agreements, self.last_proposals, self.last_utilities
        return agreements, self.last_proposals, self.last_utilities

### Loading Deep Learning Models

In [4]:
def load_models():    
    _DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
    model_dir = '../models/'
    _MODEL_URLS = {
        'mobilenetv2_coco_voctrainaug':
            'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
        'mobilenetv2_coco_voctrainval':
            'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
        'xception_coco_voctrainaug':
            'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
        'xception_coco_voctrainval':
            'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
    }
    models = dict()
    for modelname, tarname in _MODEL_URLS.items():
        download_path = os.path.join(model_dir, tarname)
        if not os.path.isfile(download_path):
            print('downloading {}, this might take a while...'.format(modelname))
            urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[modelname],
                               download_path)
            print('download completed!')
            
        else:
            print("model already present at {}".format(download_path))
        
        print('Loading model {}...'.format(modelname))
        models[modelname] = DeepLabModel.DeepLabModel(download_path)
        print('{} loaded successfully!'.format(modelname))
    return models
models = load_models()

model already present at ../models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz
Loading model xception_coco_voctrainaug...
xception_coco_voctrainaug loaded successfully!
model already present at ../models/deeplabv3_pascal_trainval_2018_01_04.tar.gz
Loading model xception_coco_voctrainval...
xception_coco_voctrainval loaded successfully!
model already present at ../models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
Loading model mobilenetv2_coco_voctrainaug...
mobilenetv2_coco_voctrainaug loaded successfully!
model already present at ../models/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
Loading model mobilenetv2_coco_voctrainval...
mobilenetv2_coco_voctrainval loaded successfully!


#### Defining label colors and names

In [5]:
LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])
FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = DeepLabModel.label_to_color_image(FULL_LABEL_MAP)

## Initializing Agents and Mediator

### Defining agent parameters

In [6]:
loaded = pd.read_csv('params.csv', index_col=0)

In [7]:

alphasliders = {m:[FloatSlider(description=l, min=0., max=1., step=0.01, value=loaded.loc[m][l]) for l in LABEL_NAMES] for m in models.keys()}
sliders = [VBox([Label(model)]+alphasliders[model]) for model in alphasliders.keys()]
HBox(sliders)

In [8]:
def show_proposals(task, proposals, utilities):
        plt.figure(figsize=(16,9), dpi=300)
        # Finding all the labels that appear in the proposals for the legend
        unique_labels = np.array([], dtype=np.int)
        for p in proposals:
            unique_labels = np.union1d(unique_labels, np.unique(p))
        unique_labels = unique_labels[unique_labels!=255] # We ignore the void label (see VOC2012 docs)
                
        max_cols = len(proposals)+2

        grid_spec = gridspec.GridSpec(1, max_cols, width_ratios= [6] + [6 for i in proposals]+ [2])
        # Show input
        plt.subplot(grid_spec[0])
        plt.imshow(task)
        plt.axis('off')
        plt.title('Input')

        for p, prop in enumerate(proposals):
            plt.subplot(grid_spec[p+1])
            plt.imshow(DeepLabModel.label_to_color_image(prop).astype(np.uint8))
            plt.axis('off')
            plt.title('Model {}'.format(p+1))

        ax = plt.subplot(grid_spec[max_cols-1])
        plt.imshow(
            FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest', aspect=0.3)
        ax.yaxis.tick_right()
        plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
        plt.xticks([], [])
        ax.tick_params(width=0.0)
        plt.grid(False)
        plt.show()

def show_agreement(agreement, i):
        labels = agreement.keys()
        plt.figure(figsize=(16,9), dpi=300)
        for lpos, lab in enumerate(labels):
            agr = agreement[lab]
            plt.subplot(1, len(labels), lpos+1)
            plt.title("Agreement {} for label {}".format(i, LABEL_NAMES[lab]))
            plt.set_cmap('gray')
            plt.axis('off')
            plt.imshow(agr)
            
def show_stats(log):
    plt.figure(figsize=(16,9))
    plt.subplot(1, 2, 1)
    util = np.array(log['util']).transpose()
    plt.title("Utility")
    for a, agent_util in enumerate(util):
        plt.plot(agent_util, label="Agent {}".format(a+1))
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.title("Remaining Pixels %")
    progress = np.array(log['remaining_points'])
    progress = progress / progress[0]
    plt.plot(progress)
    

def get_remaining_points(agreement):
    stacked = np.stack([a for k, a in agreement.items()])
    return np.count_nonzero(np.logical_and(stacked!=0, stacked!=1))
    

## Demo on a random Image

In [8]:
# Download an image and start negotiation
url = 'http://farm8.static.flickr.com/7920/47425338881_de208df2ff_m.jpg'
#url = 'https://cdn.vox-cdn.com/uploads/chorus_asset/file/8327269/Screen_Shot_2017_04_12_at_3.54.13_PM.png'
#url = 'https://proxy.duckduckgo.com/iu/?u=https%3A%2F%2Fimages.fastcompany.net%2Fimage%2Fupload%2Fw_596%2Cc_limit%2Cq_auto%3Abest%2Cf_auto%2Cfl_lossy%2Ffc%2F1683481-inline-s-2-dog-tv-share-the-remote-control.jpg&f=1'
#url = 'https://www.zastavki.com/pictures/originals/2013/People_Shadows_of_people_looked_047805_.jpg'
f = urllib.request.urlopen(url)
jpeg_str = f.read()
original_im = Image.open(BytesIO(jpeg_str))

In [10]:
alpha = {m: [slider.value for slider in alphasliders[m]] for m in alphasliders.keys()}
agents = [Agent(modelname, model, alpha[modelname]) for modelname, model in models.items()]
mediator = Mediator(agents)
print("Mediator Running")
log = {'util':[], 'remaining_points':list()}
init_proposal, init_utilities = mediator.start_new_task(original_im)  
next_step = enumerate(mediator.negotiation(timeout=10000))
display(Label("Current Step"))
step_progress = IntProgress(max=1)
display(step_progress)
display(Label("Total Progress"))
pixel_progress = FloatProgress(max=1.0)
display(pixel_progress)
grid_spec = None
from ipywidgets import IntText

def run_negotiation(n):
    if n<len(log['remaining_points']):
        return
    step_progress.max = n
    for i, (agreement, proposals, utilities) in next_step:
        step_progress.value = i
        current_rp = get_remaining_points(agreement)
        log['util'].append(utilities.mean(axis=2).mean(axis=1))
        log['remaining_points'].append(current_rp)
        pixel_progress.value = 1 - current_rp/log['remaining_points'][0]
        #print("Done {} of {}, ({})%".format(current_rp, log['remaining_points'][0], pixel_progress.value))
        if i == n-1 or pixel_progress.value >= 0.99:
            show_proposals(original_im, init_proposal, init_utilities)
            show_agreement(agreement, i)
            show_proposals(original_im, proposals, utilities)
            show_stats(log)
            return

interact(run_negotiation, n=IntText(value=1));

Mediator Running
(342, 513)
(342, 513)
(342, 513)
(342, 513)


Label(value='Current Step')

IntProgress(value=0, max=1)

Label(value='Total Progress')

FloatProgress(value=0.0, max=1.0)

interactive(children=(IntText(value=1, description='n'), Output()), _dom_classes=('widget-interact',))

UsageError: Line magic function `%tensorboard` not found.


In [11]:
models['mobilenetv2_coco_voctrainaug'].run(original_im)

ValueError: Fetch argument 'logits:0' cannot be interpreted as a Tensor. ("The name 'logits:0' refers to a Tensor which does not exist. The operation, 'logits', does not exist in the graph.")

{'mobilenetv2_coco_voctrainaug': <DeepLabModel.DeepLabModel at 0x7f0556e4dac8>,
 'mobilenetv2_coco_voctrainval': <DeepLabModel.DeepLabModel at 0x7f04aa24d7b8>,
 'xception_coco_voctrainaug': <DeepLabModel.DeepLabModel at 0x7f0556e4d550>,
 'xception_coco_voctrainval': <DeepLabModel.DeepLabModel at 0x7f059c0cdeb8>}